In [1]:
import h2o
import time
import numpy as np

In [2]:
h2o.init()

H2O cluster uptime:,4 hours 56 minutes 25 seconds 7 milliseconds
H2O cluster version:,3.3.0.99999
H2O cluster name:,anqi_fu
H2O cluster total nodes:,1
H2O cluster total memory:,1.78 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321


In [5]:
# Import and parse ACS 2013 5-year DP02 demographic data
acs_orig = h2o.upload_file(path=h2o.locate("smalldata/census/ACS_13_5YR_DP02_cleaned.zip"), col_types = (["enum"] + ["numeric"]*149))
acs_orig.describe()
acs_zcta_col = acs_orig["ZCTA5"]
acs_full = acs_orig.drop("ZCTA5")


Parse Progress: [##################################################] 100%
Uploaded py9dafb469-23a2-4430-a8a5-7d042244d2f7 into cluster with 32,989 rows and 150 cols
Rows: 32,989 Cols: 150
  chunk_type        chunk_name  count  count_percentage        size  \
0         C2   2-Byte Integers    104         69.333336      6.6 MB   
1        C2S  2-Byte Fractions     23         15.333334      1.4 MB   
2         C4   4-Byte Integers     23         15.333334      2.9 MB   

   size_percentage  
0        60.121900  
1        13.299410  
2        26.578688  
                            size  number_of_rows  number_of_chunks_per_column  \
0  172.16.2.30:54321     10.9 MB           32989                            1   
1               mean     10.9 MB           32989                            1   
2                min     10.9 MB           32989                            1   
3                max     10.9 MB           32989                            1   
4             stddev        0  B     

,ZCTA5,HC01_VC03,HC01_VC04,HC01_VC05,HC01_VC06,HC01_VC07,HC01_VC08,HC01_VC09,HC01_VC10,HC01_VC11,HC01_VC12,HC01_VC13,HC01_VC14,HC01_VC17,HC01_VC18,HC01_VC21,HC01_VC22,HC01_VC26,HC01_VC27,HC01_VC28,HC01_VC29,HC01_VC30,HC01_VC31,HC01_VC32,HC01_VC36,HC01_VC37,HC01_VC38,HC01_VC39,HC01_VC40,HC01_VC41,HC01_VC43,HC01_VC44,HC01_VC45,HC01_VC46,HC01_VC47,HC01_VC48,HC01_VC52,HC01_VC53,HC01_VC54,HC01_VC55,HC01_VC56,HC01_VC57,HC01_VC58,HC01_VC62,HC01_VC63,HC01_VC65,HC01_VC66,HC01_VC67,HC01_VC68,HC01_VC70,HC01_VC71,HC01_VC72,HC01_VC76,HC01_VC77,HC01_VC78,HC01_VC79,HC01_VC80,HC01_VC81,HC01_VC85,HC01_VC86,HC01_VC87,HC01_VC88,HC01_VC89,HC01_VC90,HC01_VC91,HC01_VC92,HC03_VC95,HC03_VC96,HC01_VC100,HC01_VC101,HC01_VC105,HC01_VC106,HC01_VC108,HC01_VC109,HC01_VC111,HC01_VC112,HC01_VC114,HC01_VC115,HC01_VC119,HC01_VC120,HC01_VC121,HC01_VC122,HC01_VC123,HC01_VC124,HC01_VC125,HC01_VC126,HC01_VC130,HC01_VC131,HC01_VC132,HC01_VC133,HC01_VC134,HC01_VC135,HC01_VC136,HC01_VC140,HC01_VC141,HC01_VC142,HC01_VC146,HC01_VC149,HC01_VC150,HC01_VC151,HC01_VC154,HC01_VC155,HC01_VC156,HC01_VC160,HC01_VC161,HC01_VC162,HC01_VC163,HC01_VC164,HC01_VC165,HC01_VC166,HC01_VC170,HC01_VC171,HC01_VC172,HC01_VC173,HC01_VC174,HC01_VC175,HC01_VC176,HC01_VC177,HC01_VC178,HC01_VC179,HC01_VC180,HC01_VC181,HC01_VC185,HC01_VC186,HC01_VC187,HC01_VC188,HC01_VC189,HC01_VC190,HC01_VC191,HC01_VC192,HC01_VC193,HC01_VC194,HC01_VC195,HC01_VC196,HC01_VC197,HC01_VC198,HC01_VC199,HC01_VC200,HC01_VC201,HC01_VC202,HC01_VC203,HC01_VC204,HC01_VC205,HC01_VC206,HC01_VC207,HC01_VC208,HC01_VC209,HC01_VC210,HC01_VC211,HC01_VC212
type,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,real,real,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,real,real,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int
mins,1001.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
maxs,99929.0,43421.0,28649.0,15305.0,19802.0,11351.0,4083.0,2157.0,12898.0,7366.0,26569.0,19593.0,7810.0,17240.0,21794.0,14.53,14.53,114353.0,43421.0,19783.0,45713.0,21866.0,15961.0,4124.0,46255.0,23453.0,22038.0,1711.0,1374.0,4419.0,47542.0,23297.0,21644.0,3303.0,5493.0,6335.0,3677.0,2527.0,1000.0,1000.0,1000.0,1000.0,1000.0,7001.0,2187.0,604.0,510.0,556.0,796.0,2187.0,1368.0,1548.0,39977.0,3236.0,3221.0,16848.0,9254.0,36548.0,73169.0,24006.0,13228.0,24509.0,16996.0,6323.0,24256.0,28318.0,100.0,100.0,83817.0,12850.0,113821.0,14673.0,40691.0,2422.0,72738.0,10304.0,34581.0,9605.0,113555.0,100211.0,27326.0,16632.0,13287.0,11849.0,10227.0,3928.0,114734.0,88228.0,86214.0,70376.0,45677.0,12507.0,69424.0,69424.0,32541.0,47531.0,70775.0,12507.0,1174.0,11333.0,69424.0,3737.0,66787.0,69424.0,28823.0,48809.0,7626.0,2614.0,55018.0,2168.0,105696.0,66897.0,84946.0,54104.0,82723.0,44842.0,40436.0,26483.0,46350.0,35889.0,29294.0,12035.0,114734.0,16031.0,32701.0,3110.0,3757.0,15498.0,16337.0,12731.0,9529.0,27423.0,5776.0,10122.0,19562.0,30502.0,1790.0,14974.0,19492.0,14824.0,13310.0,3258.0,2956.0,3733.0,12233.0,8120.0,3090.0,5320.0,2373.0,46449.0
sigma,27446.5500909,5020.60026619,3324.72651895,1601.32

In [3]:
# Import and parse WHD 2014-2015 labor violations data
whd_zcta = h2o.upload_file(path=h2o.locate("smalldata/census/whd_zcta_cleaned.zip"), col_types = (["enum"]*7 + ["numeric"]*97))
whd_zcta.describe()


Parse Progress: [##################################################] 100%
Uploaded pya9dd0971-adb5-4ac6-be3a-f57ebc0d5e64 into cluster with 208,806 rows and 104 cols
Rows: 208,806 Cols: 104
  chunk_type                 chunk_name  count  count_percentage        size  \
0        C0L          Constant Integers     40         19.230770      3.1 KB   
1        CX0                Sparse Bits      7          3.365385      674  B   
2        CXI            Sparse Integers     69         33.173077    213.7 KB   
3         C1            1-Byte Integers      4          1.923077    408.1 KB   
4        C1N  1-Byte Integers (w/o NAs)      3          1.442308    308.8 KB   
5         C2            2-Byte Integers     11          5.288462      2.2 MB   
6         C4            4-Byte Integers     16          7.692308      6.4 MB   
7        C4S           4-Byte Fractions     12          5.769231      4.8 MB   
8        CXD               Sparse Reals     46         22.115383    381.4 KB   

   size_

,case_id,trade_nm,cty_nm,st_cd,zcta5_cd,naic_cd,flsa_repeat_violator,case_violtn_cnt,cmp_assd_cnt,ee_violtd_cnt,bw_atp_amt,ee_atp_cnt,flsa_violtn_cnt,flsa_bw_atp_amt,flsa_ee_atp_cnt,flsa_mw_bw_atp_amt,flsa_ot_bw_atp_amt,flsa_15a3_bw_atp_amt,flsa_cmp_assd_amt,sca_violtn_cnt,sca_bw_atp_amt,sca_ee_atp_cnt,mspa_violtn_cnt,mspa_bw_atp_amt,mspa_ee_atp_cnt,mspa_cmp_assd_amt,h1b_violtn_cnt,h1b_bw_atp_amt,h1b_ee_atp_cnt,h1b_cmp_assd_amt,fmla_violtn_cnt,fmla_bw_atp_amt,fmla_ee_atp_cnt,fmla_cmp_assd_amt,flsa_cl_violtn_cnt,flsa_cl_minor_cnt,flsa_cl_cmp_assd_amt,dbra_cl_violtn_cnt,dbra_bw_atp_amt,dbra_ee_atp_cnt,h2a_violtn_cnt,h2a_bw_atp_amt,h2a_ee_atp_cnt,h2a_cmp_assd_amt,flsa_smw14_violtn_cnt,flsa_smw14_bw_amt,flsa_smw14_ee_atp_cnt,cwhssa_violtn_cnt,cwhssa_bw_amt,cwhssa_ee_cnt,osha_violtn_cnt,osha_bw_atp_amt,osha_ee_atp_cnt,osha_cmp_assd_amt,eppa_violtn_cnt,eppa_bw_atp_amt,eppa_ee_cnt,eppa_cmp_assd_amt,h1a_violtn_cnt,h1a_bw_atp_amt,h1a_ee_atp_cnt,h1a_cmp_assd_amt,crew_violtn_cnt,crew_bw_atp_amt,crew_ee_atp_cnt,crew_cmp_assd_amt,ccpa_violtn_cnt,ccpa_bw_atp_amt,ccpa_ee_atp_cnt,flsa_smwpw_violtn_cnt,flsa_smwpw_bw_atp_amt,flsa_smwpw_ee_atp_cnt,flsa_hmwkr_violtn_cnt,flsa_hmwkr_bw_atp_amt,flsa_hmwkr_ee_atp_cnt,flsa_hmwkr_cmp_assd_amt,ca_violtn_cnt,ca_bw_atp_amt,ca_ee_atp_cnt,pca_violtn_cnt,pca_bw_atp_amt,pca_ee_atp_cnt,flsa_smwap_violtn_cnt,flsa_smwap_bw_atp_amt,flsa_smwap_ee_atp_cnt,flsa_smwft_violtn_cnt,flsa_smwft_bw_atp_amt,flsa_smwft_ee_atp_cnt,flsa_smwl_violtn_cnt,flsa_smwl_bw_atp_amt,flsa_smwl_ee_atp_cnt,flsa_smwmg_violtn_cnt,flsa_smwmg_bw_atp_amt,flsa_smwmg_ee_atp_cnt,flsa_smwsl_violtn_cnt,flsa_smwsl_bw_atp_amt,flsa_smwsl_ee_atp_cnt,eev_violtn_cnt,h2b_violtn_cnt,h2b_bw_atp_amt,h2b_ee_atp_cnt,sraw_violtn_cnt,sraw_bw_atp_amt,sraw_ee_atp_cnt
type,int,enum,enum,enum,int,int,enum,int,real,int,real,int,int,real,int,real,real,real,real,int,real,int,int,real,int,real,int,real,int,real,int,real,int,int,int,int,real,int,real,int,int,real,int,real,int,real,int,int,real,int,int,int,int,real,int,real,int,int,int,real,int,int,int,int,int,int,int,real,int,int,real,int,int,real,int,int,int,real,int,int,real,int,int,int,int,int,real,int,int,int,int,int,int,int,int,real,int,int,int,real,int,int,int,int
mins,1918.0,0.0,0.0,0.0,601.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
maxs,1081028538.0,178147.0,11043.0,55.0,99929.0,928120.0,3.0,530002.0,3129900.0,76664.0,30857204.68,76664.0,76664.0,30857204.68,76664.0,1426727.38,30857204.68,103847.31,708950.0,13667.0,18598525.86,7170.0,905.0,123330.54,903.0,97225.0,1742.0,4222146.35,1046.0,1740000.0,530002.0,118250.0,105.0,100.0,766.0,764.0,287980.0,752.0,5745385.3,747.0,2661.0,2338699.6,1365.0,3129900.0,544.0,299592.9,544.0,6171.0,3213935.37,2153.0,32.0,0.0,0.0,37800.0,166.0,20750.78,2.0,274000.0,119.0,1041759.03,119.0,40000.0,2.0,0.0,0.0,165000.0,27.0,25000.0,27.0,220.0,188895.49,191.0,43.0,15945.66,33.0,500.0,45.0,25973.23,45.0,224.0,360000.0,224.0,0.0,0.0,0.0,7.0,1254.91,7.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,382.05,2.0,1.0,194.0,93507.0,159.0,0.0,0.0,0.0
sigma,2365752.28505,NaN,NaN,NaN,29328.3363367,269790.387236,NaN,1404.55540514,10478.9730611,198.031362337,105051.011105,197.44507725,199.164923126,84369.3689394,196.028331856,12170.227887,83106.5198098,292.690108981,4025.87848901,37.0939638198,51171.7383402,21.1854369089,5.91730846019,804.548388162,4.89925331827,695.08595789,4.94015631436,14265.1911479,3.02683292733,4924.41951758,1389.35239512,981.716454624,0.34951786496,0.437678676844,2.62657240956,2.39294258144,1912.45383029,5.83030412801,19590.9444798,3.88372599246,13.5237149577,5981.25964728,5.80482948444,8009.07651465,6.0943572942,1455.21211713,5.77363187357,15.8263

In [8]:
# Create validation data with 20% missing entries
acs_miss = h2o.upload_file(path=h2o.locate("smalldata/census/ACS_13_5YR_DP02_cleaned.zip"), col_types = (["enum"] + ["numeric"]*149))
acs_miss = acs_miss.drop("ZCTA5")
acs_miss = acs_miss.insert_missing_values(fraction=0.2)
acs_miss.describe()


Parse Progress: [##################################################] 100%
Uploaded py0853f40e-9ee7-4cae-9ac0-26a3469bc4a4 into cluster with 32,989 rows and 150 cols

Insert Missing Values Progress: [##################################################] 100%
Rows: 32,989 Cols: 149
  chunk_type        chunk_name  count  count_percentage        size  \
0         C2   2-Byte Integers    104         69.798660      6.6 MB   
1        C2S  2-Byte Fractions     23         15.436241      1.4 MB   
2         C4   4-Byte Integers     22         14.765101      2.8 MB   

   size_percentage  
0        60.824787  
1        13.454895  
2        25.720316  
                            size  number_of_rows  number_of_chunks_per_column  \
0  172.16.2.30:54321     10.8 MB           32989                            1   
1               mean     10.8 MB           32989                            1   
2                min     10.8 MB           32989                            1   
3                max     10

,HC01_VC03,HC01_VC04,HC01_VC05,HC01_VC06,HC01_VC07,HC01_VC08,HC01_VC09,HC01_VC10,HC01_VC11,HC01_VC12,HC01_VC13,HC01_VC14,HC01_VC17,HC01_VC18,HC01_VC21,HC01_VC22,HC01_VC26,HC01_VC27,HC01_VC28,HC01_VC29,HC01_VC30,HC01_VC31,HC01_VC32,HC01_VC36,HC01_VC37,HC01_VC38,HC01_VC39,HC01_VC40,HC01_VC41,HC01_VC43,HC01_VC44,HC01_VC45,HC01_VC46,HC01_VC47,HC01_VC48,HC01_VC52,HC01_VC53,HC01_VC54,HC01_VC55,HC01_VC56,HC01_VC57,HC01_VC58,HC01_VC62,HC01_VC63,HC01_VC65,HC01_VC66,HC01_VC67,HC01_VC68,HC01_VC70,HC01_VC71,HC01_VC72,HC01_VC76,HC01_VC77,HC01_VC78,HC01_VC79,HC01_VC80,HC01_VC81,HC01_VC85,HC01_VC86,HC01_VC87,HC01_VC88,HC01_VC89,HC01_VC90,HC01_VC91,HC01_VC92,HC03_VC95,HC03_VC96,HC01_VC100,HC01_VC101,HC01_VC105,HC01_VC106,HC01_VC108,HC01_VC109,HC01_VC111,HC01_VC112,HC01_VC114,HC01_VC115,HC01_VC119,HC01_VC120,HC01_VC121,HC01_VC122,HC01_VC123,HC01_VC124,HC01_VC125,HC01_VC126,HC01_VC130,HC01_VC131,HC01_VC132,HC01_VC133,HC01_VC134,HC01_VC135,HC01_VC136,HC01_VC140,HC01_VC141,HC01_VC142,HC01_VC146,HC01_VC149,HC01_VC150,HC01_VC151,HC01_VC154,HC01_VC155,HC01_VC156,HC01_VC160,HC01_VC161,HC01_VC162,HC01_VC163,HC01_VC164,HC01_VC165,HC01_VC166,HC01_VC170,HC01_VC171,HC01_VC172,HC01_VC173,HC01_VC174,HC01_VC175,HC01_VC176,HC01_VC177,HC01_VC178,HC01_VC179,HC01_VC180,HC01_VC181,HC01_VC185,HC01_VC186,HC01_VC187,HC01_VC188,HC01_VC189,HC01_VC190,HC01_VC191,HC01_VC192,HC01_VC193,HC01_VC194,HC01_VC195,HC01_VC196,HC01_VC197,HC01_VC198,HC01_VC199,HC01_VC200,HC01_VC201,HC01_VC202,HC01_VC203,HC01_VC204,HC01_VC205,HC01_VC206,HC01_VC207,HC01_VC208,HC01_VC209,HC01_VC210,HC01_VC211,HC01_VC212
type,int,int,int,int,int,int,int,int,int,int,int,int,int,int,real,real,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,real,real,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int
mins,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
maxs,43421.0,28649.0,15305.0,19802.0,11351.0,4083.0,1802.0,12898.0,7366.0,26569.0,19593.0,6958.0,15327.0,14852.0,14.53,14.53,114353.0,43421.0,19783.0,45289.0,21866.0,15961.0,4124.0,46255.0,23453.0,22038.0,1711.0,1310.0,4419.0,47542.0,23297.0,21093.0,3303.0,5493.0,6335.0,2894.0,2527.0,1000.0,1000.0,1000.0,1000.0,1000.0,7001.0,2187.0,604.0,510.0,556.0,796.0,2187.0,1368.0,1548.0,39977.0,3236.0,3221.0,16848.0,8655.0,32433.0,73169.0,24006.0,13228.0,22988.0,16996.0,6323.0,24256.0,28318.0,100.0,100.0,83817.0,12850.0,113821.0,14673.0,40691.0,2422.0,69942.0,10304.0,34581.0,6881.0,111916.0,100211.0,23539.0,16632.0,12687.0,11364.0,9577.0,3928.0,114734.0,88228.0,86214.0,68336.0,45677.0,9820.0,69424.0,69424.0,32541.0,40945.0,70775.0,12507.0,1174.0,11333.0,69424.0,3531.0,66787.0,69424.0,28823.0,48809.0,7626.0,2614.0,55018.0,2168.0,103957.0,63593.0,84946.0,54104.0,82723.0,44201.0,40436.0,26483.0,46350.0,23460.0,29294.0,6531.0,114734.0,14313.0,32701.0,3110.0,3757.0,13738.0,16337.0,12731.0,9529.0,27423.0,4485.0,5188.0,19562.0,30502.0,1790.0,14974.0,19492.0,14824.0,13310.0,3258.0,2956.0,3733.0,11694.0,8120.0,3090.0,5320.0,2373.0,46449.0
sigma,5015.1232927,3355.47903427,1599.83136624,2406.54083299,1110.68902504,275.57046

In [12]:
# Run GLRM to reduce ZCTA demographics to 15 archetypes
acs_best = h2o.glrm(x = acs_miss, 
                    validation_frame = acs_full, 
                    k = 15,
                    transform = "STANDARDIZE",
                    init = "PlusPlus",
                    loss = "Quadratic",
                    max_iterations = 1000,
                    regularization_x = "Quadratic",
                    regularization_y = "L1",
                    gamma_x = 0.25,
                    gamma_y = 0.5)
print acs_best


glrm Model Build Progress: [##################################################] 100%
Model Details
H2ODimReductionModel :  Beta - Generalized Low Rank Model
Model Key:  GLRM_model_python_1442515779061_8
     number_of_iterations  final_step_size  final_objective_value
0                     350         0.008973          773545.612386


ModelMetricsGLRM: glrm
** Reported on train data. **

MSE: NaN
Sum of Squared Error (Numeric): 763420.384245
Misclassification Error (Categorical): 0.0

ModelMetricsGLRM: glrm
** Reported on validation data. **

MSE: NaN
Sum of Squared Error (Numeric): 1037878.89793
Misclassification Error (Categorical): 0.0



In [15]:
# Embedding of ZCTAs into archetypes (X)
zcta_arch_x = h2o.get_frame(acs_best._model_json["output"]["loading_key"]["name"])
zcta_arch_x.head()

H2OFrame with 10 rows and 15 columns: 
      Arch1     Arch2     Arch3     Arch4     Arch5     Arch6     Arch7  \
0  0.021864 -0.048960 -0.132999  0.065706 -0.007834 -0.006353  0.012786   
1 -0.013743 -0.152830  0.068952  0.113557  1.153437 -0.007532 -0.013812   
2  0.990214 -0.079052  0.035010  0.050515  0.370765 -0.000569 -0.017949   
3  0.023890  0.001855 -0.038958 -0.007605  0.099265 -0.015832 -0.008600   
4  1.027913 -0.000456 -0.077164  0.011958  0.182680  0.000203 -0.006037   
5  0.005147 -0.006040 -0.002621 -0.003239  0.017930 -0.015948  0.003181   
6 -0.007146 -0.001989 -0.009491  0.004067 -0.009203 -0.016102 -0.004614   
7  0.022791 -0.009563 -0.004576  0.005622  0.034344 -0.015484  0.005543   
8  0.005807 -0.005460 -0.000457  0.000708  0.008951 -0.015111 -0.001644   
9  0.007159 -0.005659 -0.000084 -0.002319  0.027990 -0.016246 -0.000470   

      Arch8     Arch9    Arch10    Arch11    Arch12    Arch13    Arch14  \
0  0.057113  0.956857  0.091362 -0.014270 -0.110272 -0.07123

In [16]:
# Archetypes to full feature mapping (Y)
arch_feat_y = acs_best._model_json["output"]["archetypes"]
print arch_feat_y

            hc01_vc03  hc01_vc04  hc01_vc05  hc01_vc06  hc01_vc07  hc01_vc08  \
0    Arch1   0.142376   0.152304   0.108593   0.202196   0.127284   0.072222   
1    Arch2   2.729887   3.306421   3.968837   2.816958   3.595626   4.902037   
2    Arch3   2.033773   2.411468   2.508946   2.791926   3.243633   2.158417   
3    Arch4   2.011233   1.703929   1.870699   0.844545   0.813108   2.657793   
4    Arch5   2.454583   1.805393   1.684670   2.036364   1.990720   1.261040   
5    Arch6   1.833848   2.181618   0.165976   2.513753   4.045241   2.243533   
6    Arch7   2.697094   2.630092   2.219212   1.558967   1.303389   3.138458   
7    Arch8   1.802735   1.807938   1.994152   1.243351   1.326064   2.589003   
8    Arch9   1.033623   1.023104   0.937945   1.243389   1.143676   0.533788   
9   Arch10   4.134525   4.310260   3.572403   4.307610   4.048286   3.716774   
10  Arch11  -0.544742  -0.538190  -0.485494  -0.541558  -0.468660  -0.436118   
11  Arch12   2.254777   2.338267   2.240

In [4]:
# Split WHD data into test/train with 20/80 ratio
split = whd_zcta["flsa_repeat_violator"].runif()
train = whd_zcta[split <= 0.8]
test = whd_zcta[split > 0.8]

# Build a GBM model to predict repeat violators and score
s = time.time()
gbm_orig = h2o.gbm(x = train.drop("flsa_repeat_violator"),
                   y = train["flsa_repeat_violator"],
                   validation_x = test.drop("flsa_repeat_violator"),
                   validation_y = test["flsa_repeat_violator"],
                   ntrees = 10, 
                   max_depth = 6, 
                   distribution = "multinomial")
orig_elapsed = time.time() - s


gbm Model Build Progress: [##################################################] 100%


In [10]:
# Replace zcta5_cd column in WHD data with GLRM archetypes
zcta_arch_x["zcta5_cd"] = acs_zcta_col
whd_arch = whd_zcta.merge(zcta_arch_x, allLeft = True, allRight = False)
whd_arch = whd_arch.drop("zcta5_cd")
whd_arch.describe()

SyntaxError: keyword can't be an expression (<ipython-input-10-fd258a35d16a>, line 3)

In [11]:
# Split WHD data into test/train with 20/80 ratio
train_mod = whd_arch[split <= 0.8]
test_mod = whd_arch[split > 0.8]

# Build a GBM model to predict repeat violators and score
s = time.time()
gbm_mod = h2o.gbm(x = train_mod.drop("flsa_repeat_violator"),
                   y = train_mod["flsa_repeat_violator"],
                   validation_x = test_mod.drop("flsa_repeat_violator"),
                   validation_y = test_mod["flsa_repeat_violator"],
                   ntrees = 10, 
                   max_depth = 6, 
                   distribution = "multinomial")
mod_elapsed = time.time() - s

NameError: name 'whd_arch' is not defined

In [ ]:
# Model performance comparison
train_auc_orig = gbm_orig.model_performance(train).auc()
test_auc_orig  = gbm_orig.model_performance(test ).auc()
train_auc_mod  = gbm_mod .model_performance(train).auc()
test_auc_mod   = gbm_orig.model_performance(test ).auc()

# Print results in pretty HTML table
header = ["Metric"   , "Original"    , "Reduced"    ]
table = [
         ["Runtime"  , orig_elapsed  , mod_elapsed  ]
         ["Train AUC", train_auc_orig, train_auc_mod]
         ["Test AUC" , test_auc_orig , test_auc_mod ]
        ]
h2o.H2ODisplay(table,header)